<div align="center">
Solution duTravail pratique #2<br>

## Entraînement et mesure de performance d’apprentissage<br>
<div align="center">
Dans le cadre du cours 8IAR403 – apprentissage automatique pour la science des données<br><br>
Le 10 mars 2025<br><br>
</div></div>

***

L'ensemble de données exploré dans ce second travail pratique est issu d'un site de commerce électronique, représentant les revenus
générés par les clients.

Cette analyse vise à explorer les étapes et paramétrage d'un modèle d'entraînement,  et de mesure de performances d'un modèle, en exploitant
deux types de classeurs:

* Un classeur binaire appliqué à un ensemble de données possédant deux classes.
* Un classeur multi-classe appliqué à un ensemble de données possédant trois classes.

Ces deux évaluations permettront de comparer la performance de chacun des classeurs et d'identifier le modèle et les
hyper-paramètres les plus appropiés, ie. moins de surajustement.

_Il est à noter que cette analyse complète celle produite lors du travail pratique #1. Ainsi, nous réutiliserons
les mêmes approche de nettoyage et préparation des données, appliqués sous forme de pipelines de transformation permettant de générer notre TBA._ 



# Initialisation et déclaration des fonctions<a name="initialisation"></a>
Cette première section permet l'initialisation des paramètres essentiels au bon fonctionnement du projet et
définit les différentes fonctions utilisées dans le cadre de celui-ci.

### Importation des bibliothèques pour l'exécution d'un projet d'apprentissage machine

Nous débutons par l'importation des bibliothèques nécessaires pour ce projet d'apprentissage machine.
Les bibliothèques utilisées sont:
1. __pandas__, permettant l'importation et la gestion des ensembles de données nécessaires au projet.
2. __numpy__, permettant les manipulations, évaluations et fonctions mathématiques avancées sur les données.
3. __os__, permettant la gestion d'emplacement des fichiers.
4. __matplotlyb__, pour les éléments de visualisation.
5. __sklearn__, pour plusieurs opérations de traitement, mises à l'échelle et de transformation dans un contexte de ML.

In [1]:
# Importation des bibliothèques
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, StratifiedKFold
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, recall_score, precision_score, f1_score, make_scorer, roc_curve, auc, accuracy_score
from sklearn import metrics
#from sklearn.ensemble import RandomForestClassifier
pd.options.mode.chained_assignment = None

### Paramètres utilisateurs

Cette section vise à initialiser les paramètres utilisateurs. Plus précisément, dans le cadre de ce travail pratique,
ces paramètres correspondent aux noms et emplacements des fichiers de données, ainsi qu'aux ratios de division des données.

In [2]:
#Emplacement des fichiers
filepath_dataset = "Datasets"

#Noms des fichiers
filename_customer = "Customer.csv"
filename_country_pop = "CountryPopulation.csv"
filename_country_gdp = "CountryGDP.csv"

# Ratio des données de test par rapport aux données d'entrainement
test_train_ratio = 0.2

# Seed permettant de reproduire la distribution aléatoire
random_state_seed = 42

# Inclusion ou non du POP et GDP dans le merge

sans_merge = False
merge_gdp = False

### Préparation de la TBA<a name="preparation"></a>

La présente analyse complète celle produite lors du travail pratique #1. Ainsi, nous réutiliserons
les mêmes approches de nettoyage et préparation des données, appliqués sous forme de pipelines de transformation permettant
de générer notre TBA. Afin d'accélérer l'exécution, cette étape de préparation de la TBA détaille chacune des fonctions
uniquement sous forme de commentaires dans le code.

In [3]:
#### Fonction de bornage des données aberrantes (outliers)
def nettoyageOutliers(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.columns

    #Calcul de l'écart interquartile
    Q25 = df[cols].quantile(0.25)  #Q1 on définit le premier quartile pour c
    Q75 = df[cols].quantile(0.75)
    IQR = Q75 - Q25

    #Calcul des bornages inférieurs et supérieurs à l'IQR
    SeuilMin = (Q25 - 1.5 * IQR)
    SeuilMax = (Q75 + 1.5 * IQR)

    # Remplacer les outliers à l'extérieur des bornes IQR par les valeurs correspondantes
    nouv_df = df[cols].clip(SeuilMin[cols], SeuilMax[cols], axis=1)
    return nouv_df

def bornage(data):
    return nettoyageOutliers(pd.DataFrame(data))

#### Fonction de concaténation (merge) des ensembles de données
def mergeDataset(data: pd.DataFrame, pop=False, pib=False):

    if pop == True:
        #aucune jointure
        return data
    
    data_enrichie = pd.merge(data, data_pays)

    if pib == True:
        #jointure entre le data et le data_pib
        data_enrichie = data_enrichie.merge(data_pib)
    return data_enrichie

#### Fonction de conversion de caractéristiques au format numérique
def toNum(data: pd.DataFrame):
    data['first_item_prize'] = pd.to_numeric(data['first_item_prize'])
    return data

### Pipeline de traitement des données numériques
num_transformer = Pipeline([
    ('toNum', FunctionTransformer(toNum, validate=False)),
    # Nettoyage par remplacement de valeurs manquantes
    ('imputer', SimpleImputer(strategy="median")),
    # Remplacement des données aberrantes
    ('clamp', FunctionTransformer(bornage, validate=False))
])

### Pipeline de traitement des données catégorielles
cat_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

### Pipeline de tranformation des données numériques et catégorielles
preparationData = ColumnTransformer(
    transformers=[
        ('num', num_transformer, make_column_selector(dtype_include=np.number)),
        ('cat', cat_transformer, make_column_selector(dtype_exclude=np.number))
    ]
)

### Pipeline complet
full_pipeline = Pipeline([
    ('merge', FunctionTransformer(mergeDataset, kw_args={"pop":sans_merge, "pib": merge_gdp}, validate=False)),
    ('preparation', preparationData),
    ('standard_scaler', StandardScaler())
])

### Chargement des ensembles de données
Chargement des trois ensembles de données (Customer, Population et PIB) en spécifiant le format des données manquantes et
en s'assurant de faire correspondre le nom des colonnes pour les opérations de concaténation.

In [4]:
# Importation du fichier de données customer
data_client = pd.read_csv(os.path.join(filepath_dataset, filename_customer), na_values=['?', 'unknown'])

# Importation du fichier de données Population par pays et modification du nom de colonne "Country"
data_pays = pd.read_csv(os.path.join(filepath_dataset, filename_country_pop), na_values=['?', 'unknown'])
data_pays.columns = ['country', 'population']

# Importation du fichier de données PIB et modificationd de la colonne "Country"
data_pib = pd.read_csv(os.path.join(filepath_dataset, filename_country_gdp), na_values=['?', 'unknown'])
data_pib.columns = ['country', 'GDP_inhab']

### Séparation de données d'entraînement et d'évaluation

Afin d'éviter d'introduire des biais dans l'évaluation de nos données, le jeu de données est immédiatement scindé en
données d'entraînement et données de test.

In [5]:
# division des ensemble de données.
train_set, test_set = train_test_split(data_client, test_size=test_train_ratio, random_state=random_state_seed)

# Validation de la taille des ensembles de données d'entrainement et de test
print("Taille de l'ensemble de données original:\n", data_client.shape)
print("Taille de l'ensemble de données d'entraînement:\n", train_set.shape)
print("Taille de l'ensemble de données d'évaluation:\n", test_set.shape)

# Comme la variable revenue est retirée du jeu de données avant d'exécuter le pipeline, les NaN sont remplacés directement
train_set["revenue"].replace(np.nan, train_set["revenue"].median(), inplace=True)
test_set["revenue"].replace(np.nan, test_set["revenue"].median(), inplace=True)

Taille de l'ensemble de données original:
 (10000, 8)
Taille de l'ensemble de données d'entraînement:
 (8000, 8)
Taille de l'ensemble de données d'évaluation:
 (2000, 8)


***

# Question 2.1 - Entraînement d’un classeur binaire<a name="classbinaire"></a>

Dans un premier temps, on souhaite créer un classeur binaire dont la variable cible est le revenu. Il s’agit de prédire
si un client est susceptible ou non de générer plus de revenus que la moyenne.


## Question 2.1.1 - Transformation binaire de la variable "_revenue_"<a name="binarisation"></a>

Fonction permettant de remplacer la valeur de la variable "_revenue_" par rapport à la moyenne:
* Valeur 1: _Revenue_ supérieur à la moyenne
* Valeur 0: _Revenue_ inférieur ou égal à la moyenne.

In [6]:
def binarize_att (data: pd.DataFrame, att_name):

    att_mean = data[att_name].mean()
    # 1 si supérieur à la moyenne, 0 si inférieur
    #data[att_name] = (data[att_name] > att_mean).astype(int)
    data[att_name] = np.where(data[att_name] > att_mean, 1, 0)
    return data

# Transformation de l'attribut revenue en classe binaire pour les ensembles d'entrainement et de test
train_set_bin = binarize_att(train_set.copy(), "revenue")
test_set_bin = binarize_att(test_set.copy(), "revenue")

print("Distribution du revenue en fonction de la moyenne:")
print("0: Plus petit ou égal, 1: Plus grand que la moyenne")
print(train_set_bin["revenue"].value_counts())

Distribution du revenue en fonction de la moyenne:
0: Plus petit ou égal, 1: Plus grand que la moyenne
revenue
0    4865
1    3135
Name: count, dtype: int64


### Retrait de l'attribut de classification "_revenue_"

Afin d'exploiter l'arbre binaire de classification, l'attribut de classification "_revenue_" est retirée de l'ensemble
de données, le but étant de mesurer avec quelle précision le modèle pourra la prédire.

Les valeurs de l'attribut sont toutefois stockées dans les variables _Y\_train_ et _Y\_test_ afin d'évaluer la précision du modèle dans une étape subséquente.

In [7]:
### Retrait de la variable revenue de l'ensemble d'entrainement
X_train = train_set_bin.drop(['revenue'], axis=1)
Y_train = train_set_bin['revenue']

### Retrait de la variable revenue de l'ensemble de test
X_test = test_set_bin.drop(['revenue'],axis =1)
Y_test = test_set_bin['revenue']

### Application des pipelines

Les données préparées et l'attribut de classification retiré, les données sont transformés par l'application des
différents pipelines configurés à l'étape [Préparation de la TBA](#preparation)

In [8]:
#considerer uniquement le dataset Customer. Scénario1
sans_merge = True
X_train_nopop = full_pipeline.fit_transform(X_train)
X_test_nopop = full_pipeline.fit_transform(X_test)



# Pipeline appliqué aux données, sans le PIB. Scenario 2
merge_gdp = False
X_train_nogdp = full_pipeline.fit_transform(X_train)
X_test_nogdp = full_pipeline.fit_transform(X_test)


# Pipeline appliqué aux données, avec le PIB. Scenario 3
merge_gdp = True
X_train_gdp = full_pipeline.fit_transform(X_train)
X_test_gdp = full_pipeline.fit_transform(X_test)

/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/pyt

## Question 2.1.2 - Échantillonnage aléatoire stratifié<a name="sampling"></a>

Nous procédons donc à l'échantillonnage aléatoire stratifié de nos ensembles de données selon des échantillons de taille 2000, 4000 et 8000.

Chacun des échantillons est généré afin d'inclure ou non le PIB, en fonction de la valeur de la variable _merge\_gdp_ passée au _pipeline_.

In [9]:
#Dictionnaire des tailles d'échantillonnage aléatoire. Seul les trois premières valeurs sont retenues.
sample_size = [2000, 4000, 8000]

sans_merge = True
#Création des jeux d'entrainement (X_train) et de validation (Y_train) pour le scénario 1
X_train_nopop_samp1, Y_train_nopop_samp1 = resample(X_train_nopop, Y_train, n_samples=sample_size[0], random_state=random_state_seed, stratify=Y_train)
X_train_nopop_samp2, Y_train_nopop_samp2 = resample(X_train_nopop, Y_train, n_samples=sample_size[1], random_state=random_state_seed, stratify=Y_train)
X_train_nopop_samp3, Y_train_nopop_samp3 = resample(X_train_nopop, Y_train, n_samples=sample_size[2], random_state=random_state_seed, stratify=Y_train)



#Création des jeux d'entrainement (X_train) et de validation (Y_train) sans PIB : scenario 2

merge_gdp = False
X_train_nogdp_samp1, Y_train_nogdp_samp1 = resample(X_train_nogdp, Y_train, n_samples=sample_size[0], random_state=random_state_seed, stratify=Y_train)
X_train_nogdp_samp2, Y_train_nogdp_samp2 = resample(X_train_nogdp, Y_train, n_samples=sample_size[1], random_state=random_state_seed, stratify=Y_train)
X_train_nogdp_samp3, Y_train_nogdp_samp3 = resample(X_train_nogdp, Y_train, n_samples=sample_size[2], random_state=random_state_seed, stratify=Y_train)


#Création des jeux d'entrainement (X_train) et de validation (Y_train) avec PIB : scenario 3
merge_gdp = True

X_train_gdp_samp1, Y_train_gdp_samp1 = resample(X_train_gdp, Y_train, n_samples=sample_size[0], random_state=random_state_seed, stratify=Y_train)
X_train_gdp_samp2, Y_train_gdp_samp2 = resample(X_train_gdp, Y_train, n_samples=sample_size[1], random_state=random_state_seed, stratify=Y_train)
X_train_gdp_samp3, Y_train_gdp_samp3 = resample(X_train_gdp, Y_train, n_samples=sample_size[2], random_state=random_state_seed, stratify=Y_train)


## Question 2.1.3 - Entraînement et évaluation par validation croisée<a name="TrainEval"></a>

La première étape consiste à initialiser les arbres de décision, pour ensuite introduire les données de chacun via la méthode _fit_. Comme décrit à la section [2.1.2](#sampling), des arbres sont créés pour chacun des scénarios, soit l'ensemble d'entraînement complet (8000 observations) et des échantillons de 1000, 2000 et 4000 instances. 

Chacun des scénarios est exploré avec et sans jointure du PIB.

In [10]:
# Définition du classeur. 
clf = DecisionTreeClassifier(criterion='entropy')


### Évaluation croisée

L'évaluation croisée des résultats étant nécessaire pour chacun des arbres créés à l'étape précédente, celle-ci est présentée sous forme de fonction.

Cette fonction repose principalement sur la méthode _cross_val_score_ permettant de retourner les métriques de précision, rappel et f1. 

In [11]:
def cvs_wrapper(model, X_data, Y_data, value_count, scenario):

    kfold = [3,10]

    for fold in kfold:

        scores =[]
        scores = cross_val_score(model, X_data, Y_data, cv=fold, scoring="precision")
        scores_recall = cross_val_score(model, X_data, Y_data, cv=fold, scoring="recall")
        scores_f1 = cross_val_score(model, X_data,Y_data,cv=fold, scoring="f1")
        scores_accuracy = cross_val_score(model, X_data,Y_data,cv=fold, scoring="accuracy")

        scores = pd.DataFrame(scores, columns=['precision'])
        scores['recall'] = scores_recall
        scores['F1'] = scores_f1
        scores['accuracy']=scores_accuracy

        print("Scores de validation croisée pour un échantillon de", value_count, "instances,", fold, "plis (folds), sur des données", scenario, ":")
        print(scores.sort_values(by=['F1'], ascending=False))
        print("\n")

La fonction créée, elle est appelée pour chacune des échantillons de données. Les résultats sont affichés dans les tableaux-ci dessous.

In [12]:
#scenario #1
cvs_wrapper(clf, X_train_nopop_samp1, Y_train_nopop_samp1, sample_size[0], "scenario1")
cvs_wrapper(clf, X_train_nopop_samp2, Y_train_nopop_samp2, sample_size[1], "scenario1")
cvs_wrapper(clf, X_train_nopop_samp3, Y_train_nopop_samp3, sample_size[2], "scenario1")


#scenario #2
print("\nPerformance des modèles scenario2****************************************")
cvs_wrapper(clf, X_train_nogdp_samp1, Y_train_nogdp_samp1, sample_size[0], "scenario2")
cvs_wrapper(clf, X_train_nogdp_samp2, Y_train_nogdp_samp2, sample_size[1], "scenario2")
cvs_wrapper(clf, X_train_nogdp_samp3, Y_train_nogdp_samp3, sample_size[2], "scenario2")

#scenario #3
print("\nPerformance des modèles scenario3****************************************")
cvs_wrapper(clf, X_train_gdp_samp1, Y_train_gdp_samp1, sample_size[0], "scenario3")
cvs_wrapper(clf, X_train_gdp_samp2, Y_train_gdp_samp2, sample_size[1], "scenario3")
cvs_wrapper(clf, X_train_gdp_samp3, Y_train_gdp_samp3, sample_size[2], "scenario3")


Scores de validation croisée pour un échantillon de 2000 instances, 3 plis (folds), sur des données scenario1 :
   precision    recall        F1  accuracy
2   0.523636  0.540230  0.527103  0.629129
0   0.521898  0.553435  0.527103  0.626687
1   0.494624  0.490421  0.496296  0.601199


Scores de validation croisée pour un échantillon de 2000 instances, 10 plis (folds), sur des données scenario1 :
   precision    recall        F1  accuracy
0   0.582278  0.620253  0.587500     0.685
4   0.534884  0.576923  0.580247     0.650
3   0.560000  0.518987  0.565789     0.660
9   0.571429  0.538462  0.563758     0.650
1   0.532609  0.607595  0.559524     0.635
8   0.577465  0.500000  0.529801     0.665
6   0.477778  0.576923  0.523256     0.575
5   0.525641  0.500000  0.516129     0.625
2   0.475610  0.518987  0.496894     0.575
7   0.440000  0.487179  0.438710     0.575


Scores de validation croisée pour un échantillon de 4000 instances, 3 plis (folds), sur des données scenario1 :
   precision  

### Résultats

Nous constatons donc que le meilleur résultat provient du modèle analysé selon les paramètres: 

* Échantillon de 8000 instances
* 10 plis (folds)
* Données sans pop et pip

Les résultats sont: 
* precision: 0.75
* recall: 0.79 
* F1: 0.77
* accuracy : 0.82


## Question 2.1.5 - Optimisation des hyper-paramètres avec GridSearch<a name="gridsearch"></a>

Cette fonction tire profit de la méthode _GridSearchCV_ afin d'estimer les meilleurs paramètres pour l'ensemble de données sélectionné.



In [13]:
folds = 3
params_grid = {'max_depth': list(range(1,20)), 'min_samples_split': [10, 15, 20, 25]}


clf = DecisionTreeClassifier(criterion='entropy')

def grid_search_wrapper(splits, x_train, y_train):

    skf = StratifiedKFold(n_splits = splits)
    gs = GridSearchCV(clf, params_grid, cv=skf, return_train_score=True, n_jobs=-1)
    gs.fit(x_train, y_train)
    #results=pd.DataFrame(gs.cv_results_)
    #print(results['mean_test_score'])
    
    
    return gs

Nous débutons le reglage du modele du scenario1

In [14]:
grid_search_nopop = grid_search_wrapper(folds, X_train_nopop, Y_train)

print("scenario1")
print('Meilleurs paramètres')
print(grid_search_nopop.best_params_)


scenario1
Meilleurs paramètres
{'max_depth': 2, 'min_samples_split': 10}


Nous poursuivons le reglage du modele du scenario2

In [15]:
grid_search_nogdp = grid_search_wrapper(folds, X_train_nogdp, Y_train)
print("scenario2")
print('Meilleurs paramètres')
print(grid_search_nogdp.best_params_)

scenario2
Meilleurs paramètres
{'max_depth': 2, 'min_samples_split': 10}


Nous ajoutons le reglage du modele du scenario3

In [16]:
grid_search_gdp = grid_search_wrapper(folds, X_train_gdp, Y_train)
print("scenario3")
print('Meilleurs paramètres')
print(grid_search_gdp.best_params_)

scenario3
Meilleurs paramètres
{'max_depth': 2, 'min_samples_split': 10}


# Question 2.1.6 - Test du meilleur modèle<a name="testmodel">
    
Les paramètres ayant été identifiés, nous évaluons les résultats sur l'ensemble de données de test.  

In [17]:
y_pred = grid_search_nopop.predict(X_test_nopop)
print("test avec le scenario1")
print('\nMatrice de confusion pour Decision Tree Classifier pour le jeu de données test:')
print(pd.DataFrame(confusion_matrix(Y_test, y_pred),columns=['pred_neg', 'pred_pos'],index = ['neg', 'pos']))

print(metrics.classification_report(Y_test,y_pred))

test avec le scenario1

Matrice de confusion pour Decision Tree Classifier pour le jeu de données test:
     pred_neg  pred_pos
neg      1242         7
pos       749         2
              precision    recall  f1-score   support

           0       0.62      0.99      0.77      1249
           1       0.22      0.00      0.01       751

    accuracy                           0.62      2000
   macro avg       0.42      0.50      0.39      2000
weighted avg       0.47      0.62      0.48      2000



In [18]:
#nous poursuivons avec le test avec le scenario2

In [19]:
#test avec le scenario2
print("test avec le scenario2")
y_pred = grid_search_nogdp.predict(X_test_nogdp)
print('\nMatrice de confusion pour Decision Tree Classifier pour le jeu de données test:')
print(pd.DataFrame(confusion_matrix(Y_test, y_pred),columns=['pred_neg', 'pred_pos'],index = ['neg', 'pos']))

print(metrics.classification_report(Y_test,y_pred))

test avec le scenario2

Matrice de confusion pour Decision Tree Classifier pour le jeu de données test:
     pred_neg  pred_pos
neg      1242         7
pos       749         2
              precision    recall  f1-score   support

           0       0.62      0.99      0.77      1249
           1       0.22      0.00      0.01       751

    accuracy                           0.62      2000
   macro avg       0.42      0.50      0.39      2000
weighted avg       0.47      0.62      0.48      2000



Idem avec scenario3

In [20]:
print("test avec le scenario3")

print('\nMatrice de confusion pour Decision Tree Classifier pour le jeu de données test:')
print(pd.DataFrame(confusion_matrix(Y_test, y_pred),columns=['pred_neg', 'pred_pos'],index = ['neg', 'pos']))

print(metrics.classification_report(Y_test,y_pred))

test avec le scenario3

Matrice de confusion pour Decision Tree Classifier pour le jeu de données test:
     pred_neg  pred_pos
neg      1242         7
pos       749         2
              precision    recall  f1-score   support

           0       0.62      0.99      0.77      1249
           1       0.22      0.00      0.01       751

    accuracy                           0.62      2000
   macro avg       0.42      0.50      0.39      2000
weighted avg       0.47      0.62      0.48      2000



## Question 2.1.8 - Analyse et interprétation des résultats<a name="analysis">
    
    
Les différents scénarios d'échantillonnage ont permis d'établir que le modèle lors de la validation croisée les meilleures performances: 
    
* Échantillon de 8000 instances
* 10 plis (folds)
* Donnés sans pop et pip
* precision: 0.75
* recall: 0.79 
* F1: 0.77
    
L'exploration des hyperparamètres a permis d'estimer que la meilleure combinaison de paramètres était: 
    
* _max\_depth_: 2
* _min\_samples\_split_: 10
    
La performance de l'exactitude (accuracy) obtenue avec les données de test est de l'ordre de 60% qui est inférieure à celle obtenue avec la validation croisée qui est de 82%. On peut dire que le modele surajuste les données. Une maniere de régler ce problele de sur-apprentissage est de tenter d'améliorer le modele en 1) recourir à l'ajout d'autres données, ajouter des données supplémentaires d'entrainement si disponibilité ou bien 2) régalage des hyperparametres. 
    
    

# Question 2.2 - Entraînement d’un classeur multi-classes<a name="multiclass"></a>

## 2.2.1 - Transformation multi-classe de la variable "revenue"<a name="transfoMulti"></a>



une dicrétisation basée sur les quartiles:
* _revenu\_bas_: les données plus petites ou égales au 1er quantile.
* _revenu\_moyen_: les données contenues dans le 2ième et le 3ième quartile.
* _revenu\_élevé_: les données plus grandes que le 3ième quartile.

La fonction ci-dessous permet donc de modifier l'attribut de classification selon ce processus de discrétisation.

In [21]:
def multiclass_att(data: pd.DataFrame, att_name):

    # Délimiter les quartiles
    Q1 = data[att_name].quantile(0.25)
    Q3 = data[att_name].quantile(0.75)

    # vecteur de conditions
    conditions = [
        (data[att_name] < Q1),
        (data[att_name] >= Q1) & (data[att_name] <= Q3),
        (data[att_name] > Q3)]

    # vecteur de valeurs
    conditions_values = ["revenus_bas", "revenus_moyens", "revenus_eleve"]
    
    # remplacement des valeurs
    data[att_name] = np.select(conditions, conditions_values)

    return data

Les modifications sont donc apportées à l'ensemble de données avin de modifier la classification et le diviser en ensemble d'entrainement et de validation. 

In [22]:
# discrétisation de la variable revenue en fonction des 3 catégories

train_set_multi = multiclass_att(train_set.copy(), "revenue")
print("Distribution des classes pour le jeu d'entrainement:\n",train_set_multi["revenue"].value_counts(ascending=False))

test_set_multi = multiclass_att(test_set.copy(), "revenue")
print("\nDistribution des classes pour le jeu de test:\n", test_set_multi["revenue"].value_counts(ascending=False))

### Retrait de la variable revenue de l'ensemble d'entrainement
X_train_multi = train_set_multi.drop(['revenue'], axis=1)
Y_train_multi = train_set_multi['revenue']

### Retrait de la variable revenue de l'ensemble de test
X_test_multi = test_set_multi.drop(['revenue'],axis =1)
Y_test_multi = test_set_multi['revenue']

### Application du pipeline aux deux ensemble de données
X_train_multi = full_pipeline.fit_transform(X_train_multi)
X_test_multi = full_pipeline.fit_transform(X_test_multi)

Distribution des classes pour le jeu d'entrainement:
 revenue
revenus_moyens    4014
revenus_bas       1998
revenus_eleve     1988
Name: count, dtype: int64

Distribution des classes pour le jeu de test:
 revenue
revenus_moyens    1010
revenus_bas        495
revenus_eleve      495
Name: count, dtype: int64


/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Validation croisée du multi-classe

In [23]:
clf_multi = DecisionTreeClassifier(criterion="entropy") 

cross_val_score(clf_multi, X_train_multi, Y_train_multi, cv=3, scoring="accuracy")
y_pred_multi = cross_val_predict(clf_multi, X_train_multi, Y_train_multi)


labels = ["revenus_bas", "revenus_eleve", "revenus_moyens"]

print("\nPerformance du modèle de classification multi-classe:")
print(classification_report(Y_train_multi, y_pred_multi, target_names=labels))


Performance du modèle de classification multi-classe:
                precision    recall  f1-score   support

   revenus_bas       0.25      0.27      0.26      1998
 revenus_eleve       0.23      0.24      0.23      1988
revenus_moyens       0.49      0.47      0.48      4014

      accuracy                           0.36      8000
     macro avg       0.32      0.32      0.32      8000
  weighted avg       0.37      0.36      0.36      8000



## Question 2.2.2 - Meilleur classeur pour discriminer les classes<a name="ClassMulti"></a>




In [24]:
### Grid search afin d'identifier les meilleurs paramètres
params = {'max_depth': list(range(2, 20)), 'min_samples_split': [10, 15, 20]}

gs_multi = GridSearchCV(DecisionTreeClassifier(criterion="entropy"), params, n_jobs=-1, cv=10, verbose=1)
gs_multi.fit(X_train_multi, Y_train_multi)

#le meilleur estimateur (modele) trouvé est:
print("\nMeilleur estimateur: ", gs_multi.best_estimator_)

### test du meilleur modele obtenus:
y_pred_multi = gs_multi.predict(X_test_multi)

### Évaluation des mesures de performance:

print("\nMatrice de confusion pour le classeur multi-clases:")
print(metrics.confusion_matrix(Y_test_multi,y_pred_multi))

labels = ["revenus_bas", "revenus_eleve", "revenus_moyens"]

print("\nPerformance du modèle de classification multi-classe:")
print(classification_report(Y_test_multi, y_pred_multi, target_names=labels))

Fitting 10 folds for each of 54 candidates, totalling 540 fits

Meilleur estimateur:  DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=20)

Matrice de confusion pour le classeur multi-clases:
[[   3    0  492]
 [   2    0  493]
 [   3    0 1007]]

Performance du modèle de classification multi-classe:
                precision    recall  f1-score   support

   revenus_bas       0.38      0.01      0.01       495
 revenus_eleve       0.00      0.00      0.00       495
revenus_moyens       0.51      1.00      0.67      1010

      accuracy                           0.51      2000
     macro avg       0.29      0.33      0.23      2000
  weighted avg       0.35      0.51      0.34      2000



/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/abouzoua/Desktop/enseignement/8INF867/v8inf867/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this 

## Question 2.2.3 - Comparaison des résultats avec le classeur binaire<a name="compBinMulti"></a>

Le tableau ci-dessous présente les différents résultats en fonctions des types de classeurs et des jeux de données (_train_ et _test_): 

| Paramètre           | Binaire     | Multi-classe |
|     :----:          |:----:       |:----:        |
|_max\_depth_         | 19          |4             |
|_min\_samples\_split_| 10          |10            |
||||||
| Accuracy - Train     |0.82    |0.36|
| Accuracy - Test    |0.60    |0.50|

Nous constatons que le modèle de classification binaire est plus acceptable mais il surajuste le jeu de données. Le niveau de précision de l'autre modele étant faible et sous-ajuste les données.  nous pourrions considérer: 
* Explorer davantage d'hyper-paramètres via la méthode _GridSearchCV_. Bien que ces paramètres pourraient améliorer les performances, il est peu probable qu'ils aient un impact significatif. 
* Explorer une classification selon laquelle le nombre de classes demeure le même, mais dont la distribution des données parmi celles-ci est mieux balancée, permettant d'éviter certains de biais d'entraînement liés aux nombres de données par classes.
* Revesiter certains variables comme country,
* Explorer d'autres techniques...

